# **01-Paramétrage**

***libraries and variables***



In [1]:
# @title
from google.colab import drive
drive.mount('/content/OrangeOpen')
#------------------------------------------->
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")
#------------------------------------------->
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
job_config = bigquery.LoadJobConfig(autodetect= True)

Mounted at /content/OrangeOpen


***Recuperation des bases de données***

In [2]:
# @title
df_Effectif_Global=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Effectif_Global.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Date_Table=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Date_Table.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_MT=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/MT.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')

df_Base_Reit_ND=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Reiteration_ND.csv' , delimiter=";" , decimal=",")
df_Base_Reit=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Reiteration.csv' , delimiter=";" , decimal=",")
df_Base_RT=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_RT.csv' , delimiter=";" , decimal=",")
df_Base_Commerce=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Commerce.csv' , delimiter=";" , decimal=",")
df_Base_CmdFibre=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Commande_Fibre.csv' , delimiter=";" , decimal=",")
df_Base_Data_agent_perf_Evita=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_agent_perf_Evita.csv' , delimiter=";" , decimal=",")
df_Base_Data_agent_Retrait_Evita=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_agent_Retrait_Evita.csv' , delimiter=";" , decimal=",")
df_Base_PRA=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_PRA.csv' , delimiter=";" , decimal=",")
df_Base_REAL=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_REAL.csv' , delimiter=";" , decimal=",")
df_Base_HAPPY=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Happy.csv' , delimiter=";" , decimal=",")
df_Base_DiagonalMobile=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_DiagonalMobile.csv' , delimiter=";" , decimal=",")
df_Base_Reclamation=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Reclamation.csv' , delimiter=";" , decimal=",")
df_Base_Chorus=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Chorus.csv' , delimiter=";" , decimal=",")
df_Base_Sondage_ATM=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Sondage_ATM.csv' , delimiter=";" , decimal=",")
df_Base_Cubes_Appels=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Cubes_Appels.csv' , delimiter=";" , decimal=",")
df_Base_Data_Skills_Evita=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_Skills_Evita.csv' , delimiter=";" , decimal=",")
df_Base_Flux_Quotidien=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Flux_Quotidien.csv' , delimiter=";" , decimal=",")
df_Base_AvisClient=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_AvisClient.csv' , delimiter=";" , decimal=",")
df_Base_Sondage_OPEN=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Sondage_OPEN.csv' , delimiter=";" , decimal=",")
df_Base_Commerce_Annulation=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Annulation_Commerce.csv' , delimiter=";" , decimal=",")

df_Base_PrevCapa_Open=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/PrevCapa/PrevCapa_Jour_Open.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Base_PrevCapa_Open_Intervalle=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/PrevCapa/PrevCapa_Intervalle_Open.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')

# **02-Sondage**

***Avis Client***

In [3]:
# @title
df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])

df_AvisClient_1 = df_Base_AvisClient[['Date_Format_1', 'intervenant_code' ,'Etoile_1' , 'Etoile_2' , 'Etoile_3' , 'Etoile_4' , 'Etoile_5' , 'Satisfaction_Attente' , 'Satisfaction_Acceuil' , 'Satisfaction_Ecoute' , 'Satisfaction_Solution' , 'Satisfaction_Découverte' , 'Den_Satisfaction_Découverte' , 'TEP' , 'Den_TEP' , 'Rappel' , 'Traitement' , 'Main_Equipe' , 'Non_Traite']]
df_AvisClient_1.rename(columns = {'intervenant_code':'CUID','Den_Satisfaction_Découverte':'Den_Satisfaction_Decouverte','Satisfaction_Découverte':'Satisfaction_Decouverte'}, inplace = True)

t_table_AvisClient = pd.pivot_table(df_AvisClient_1, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
df_AvisClient_1=pd.DataFrame(t_table_AvisClient.to_records())

df_AvisClient_1 = pd.merge(left=df_AvisClient_1, right=df_Date_Table, how='left' , on='Date_Format_1')
df_AvisClient_1 = pd.merge(left=df_AvisClient_1, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

DN = pd.Timestamp.now().to_period('D')
DN.to_timestamp().strftime("%Y%m%d")
df_AvisClient_1 = df_AvisClient_1[(df_AvisClient_1['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

df_AvisClient_1['Datation']=df_AvisClient_1['Datation'].values.astype(str)
df_AvisClient_1['Datation'] = df_AvisClient_1['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

df_AvisClient_1.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_AvisClient.csv' , index=False, sep = ';' , decimal=",")

Id_Project='orange-open-sondage'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_AvisClient=Id_dataset+'.Tab_AvisClient'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_AvisClient)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_AvisClient_1, table, job_config=job_config)

df_AvisClient_2 = df_Base_AvisClient[['Date_Format_1', 'intervenant_code' ,'CIDENT', 'verbatim','noteGlobale', 'Etoile_1' , 'Etoile_2' , 'Etoile_3' , 'Etoile_4' , 'Etoile_5' , 'Satisfaction_Attente' , 'Satisfaction_Acceuil' , 'Satisfaction_Ecoute' , 'Satisfaction_Solution' , 'Satisfaction_Découverte' , 'Den_Satisfaction_Découverte']]
df_AvisClient_2.rename(columns = {'intervenant_code':'CUID','Den_Satisfaction_Découverte':'Den_Satisfaction_Decouverte','Satisfaction_Découverte':'Satisfaction_Decouverte'}, inplace = True)

t_table_Verbatim = pd.pivot_table(df_AvisClient_2, index=['Date_Format_1', 'CUID' ,'CIDENT', 'verbatim','noteGlobale'],aggfunc = 'sum')
df_AvisClient_2=pd.DataFrame(t_table_Verbatim.to_records())

DN = pd.Timestamp.now().to_period('D')
DN.to_timestamp().strftime("%Y%m%d")
df_AvisClient_2 = df_AvisClient_2[(df_AvisClient_2['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

df_AvisClient_2 = pd.merge(left=df_AvisClient_2, right=df_Date_Table, how='left' , on='Date_Format_1')
df_AvisClient_2 = pd.merge(left=df_AvisClient_2, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

df_AvisClient_2['Datation']=df_AvisClient_2['Datation'].values.astype(str)
df_AvisClient_2['Datation'] = df_AvisClient_2['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-sondage'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Verbatim=Id_dataset+'.Tab_Verbatim'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Verbatim)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_AvisClient_2, table, job_config=job_config)

# **03-Traçages**

***Happy | Reclamation | Diagonale | Traçage***

In [4]:
# @title
df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])

df_Tracage_Annexe_0 = df_Base_Data_agent_perf_Evita.loc[ (df_Base_Data_agent_perf_Evita['Flux'] == 'Open') | (df_Base_Data_agent_perf_Evita['Flux'] == 'DART') ]
df_Tracage_Annexe_0 = df_Tracage_Annexe_0[['Date_Format_1', 'CUID','Appels_traites']]

df_Tracage_Annexe_1 = df_Base_HAPPY.drop(columns=['Datation','Mois','Boutique_Equipe_Dep','Carnet','IDRdv'])

df_Tracage_Annexe_2 = df_Base_DiagonalMobile.drop(columns=['Datation','LibMois','THEME','Libelle_Conclusion','TauxScript'])

df_Tracage_Annexe_3 = df_Base_Reclamation.drop(columns=['Datation','AnneeMois','Total_Tx_de_Traitees_au_N1'])
df_Tracage_Annexe_3 = df_Tracage_Annexe_3[df_Tracage_Annexe_3['Date_Format_1'] < df_Tracage_Annexe_3['Date_Format_1'].max()-1]

df_Tracage_Annexe_4 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp']]
df_Tracage_Annexe_4.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Tracage_Annexe_4['Nbre_Tracage'] = 1

df_Tracage_Annexe_5 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp','N_client']]
df_Tracage_Annexe_5.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Tracage_Annexe_5 = df_Tracage_Annexe_5.drop_duplicates(subset=['Date_Format_1', 'N_client','CUID'] , keep="first", inplace=False)
df_Tracage_Annexe_5['Nbre_Tracage_NClient_Date'] = 1
df_Tracage_Annexe_5 = df_Tracage_Annexe_5.drop(columns=['N_client'])

df_Tracage_Annexe_0.loc[df_Tracage_Annexe_0['Date_Format_1'] <= df_Tracage_Annexe_5['Date_Format_1'].max() , 'Appels_traites_Tracage'] = df_Tracage_Annexe_0['Appels_traites']

df_Tracage_Annexe_6 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp','N_client']]
df_Tracage_Annexe_6.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Tracage_Annexe_6['Mois']=df_Tracage_Annexe_6['Date_Format_1'].values.astype(str)
df_Tracage_Annexe_6['Mois'] = df_Tracage_Annexe_6['Mois'].str[:6]
df_Tracage_Annexe_6 = df_Tracage_Annexe_6.drop_duplicates(subset=['Mois', 'N_client','CUID'] , keep="first", inplace=False)
df_Tracage_Annexe_6['Nbre_Tracage_NClient_Mois'] = 1
df_Tracage_Annexe_6 = df_Tracage_Annexe_6.drop(columns=['N_client','Mois'])

df_Tracage_Annexe_7 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp','N_dossier']]
df_Tracage_Annexe_7['Nbre_Tracage'] = 1
df_Tracage_Annexe_7.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID' , 'N_dossier':'NumeroDossier'}, inplace = True)
df_Tracage_Annexe_7 = pd.concat([df_Tracage_Annexe_7, df_Base_Reit_ND], join='outer')
df_Tracage_Annexe_7 = df_Tracage_Annexe_7.drop(columns=['Datation','Reit1J','Reit1h','Reit7J'])
t_table_Tracage_Annexe_7 = pd.pivot_table(df_Tracage_Annexe_7, index=['CUID', 'Date_Format_1','NumeroDossier'],aggfunc = 'sum')
df_Tracage_Annexe_7=pd.DataFrame(t_table_Tracage_Annexe_7.to_records())
df_Tracage_Annexe_7.loc[df_Tracage_Annexe_7['Nbre_Tracage'] >= df_Tracage_Annexe_7['exploitables'] , 'Nbre_Tracage_ND'] = df_Tracage_Annexe_7['exploitables']
df_Tracage_Annexe_7.loc[df_Tracage_Annexe_7['Nbre_Tracage'] < df_Tracage_Annexe_7['exploitables'] , 'Nbre_Tracage_ND'] = df_Tracage_Annexe_7['Nbre_Tracage']
df_Tracage_Annexe_7 = df_Tracage_Annexe_7.drop(columns=['NumeroDossier','Nbre_Tracage'])
t_table_Tracage_Annexe_7 = pd.pivot_table(df_Tracage_Annexe_7, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
df_Tracage_Annexe_7=pd.DataFrame(t_table_Tracage_Annexe_7.to_records())

df_Tracage_Annexe_7.loc[df_Tracage_Annexe_7['Date_Format_1'] <= df_Tracage_Annexe_5['Date_Format_1'].max() , 'exploitables_Tracage'] = df_Tracage_Annexe_7['exploitables']

dfg_Tracage_Annexe = pd.concat([df_Tracage_Annexe_0, df_Tracage_Annexe_1, df_Tracage_Annexe_2, df_Tracage_Annexe_3, df_Tracage_Annexe_4, df_Tracage_Annexe_5, df_Tracage_Annexe_6, df_Tracage_Annexe_7], join='outer')
t_table_Tracage_Annexe = pd.pivot_table(dfg_Tracage_Annexe, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
dfg_Tracage_Annexe=pd.DataFrame(t_table_Tracage_Annexe.to_records())

dfg_Tracage_Annexe = pd.merge(left=dfg_Tracage_Annexe, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Tracage_Annexe = pd.merge(left=dfg_Tracage_Annexe, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

#dfg_Tracage_Annexe.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Tracage.csv' , index=False, sep = ';' , decimal=",")

dfg_Tracage_Annexe['Datation']=dfg_Tracage_Annexe['Datation'].values.astype(str)
dfg_Tracage_Annexe['Datation'] = dfg_Tracage_Annexe['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-tracage'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Tracage_Global=Id_dataset+'.Tab_Tracage_Global'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Tracage_Global)
bq.delete_table(Id_table_Tracage_Global, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Tracage_Annexe, Id_table_Tracage_Global, job_config=job_config)

df_Tracage_Annexe_8 = df_Base_Chorus[['Date_Format_1', 'N_client', 'N_dossier', 'Code_Alliance_Conseiller_Resp', 'Theme_Libelle', 'Resolution_Libelle', 'Conclusion_Libelle']]
df_Tracage_Annexe_8.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)

df_Tracage_Annexe_8 = pd.merge(left=df_Tracage_Annexe_8, right=df_Date_Table, how='left' , on='Date_Format_1')
df_Tracage_Annexe_8 = pd.merge(left=df_Tracage_Annexe_8, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

#df_Tracage_Annexe_8.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Tracage_Detail.csv' , index=False, sep = ';' , decimal=",")

df_Tracage_Annexe_8['Datation']=df_Tracage_Annexe_8['Datation'].values.astype(str)
df_Tracage_Annexe_8['Datation'] = df_Tracage_Annexe_8['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

#df_Tracage_Annexe_8=df_Tracage_Annexe_8.head(1)


Id_Project='orange-open-tracage'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Tracage_Detail=Id_dataset+'.Tab_Tracage_Detail'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Tracage_Detail)
bq.delete_table(Id_table_Tracage_Detail, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_Tracage_Annexe_8, Id_table_Tracage_Detail, job_config=job_config)

***Sondage ATM***

In [5]:
# @title
df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])

df_Sondage_ATM_1 = df_Base_Chorus.loc[ df_Base_Chorus['Theme_Libelle'] == '05 Assistance technique / SAV' ]
df_Sondage_ATM_1 = df_Sondage_ATM_1[['Date_Format_1', 'Code_Alliance_Conseiller_Resp']]
df_Sondage_ATM_1.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Sondage_ATM_1['Nbre_Tracage_ATM'] = 1
t_table_Sondage_ATM_1 = pd.pivot_table(df_Sondage_ATM_1, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
df_Sondage_ATM_1=pd.DataFrame(t_table_Sondage_ATM_1.to_records())

df_Sondage_ATM_2 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp']]
df_Sondage_ATM_2.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Sondage_ATM_2['Nbre_Tracage'] = 1
t_table_Sondage_ATM_2 = pd.pivot_table(df_Sondage_ATM_2, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
df_Sondage_ATM_2=pd.DataFrame(t_table_Sondage_ATM_2.to_records())

df_Sondage_ATM_3 = df_Base_Sondage_ATM[['Date_Format_1', 'CUID','NumerateurSco','DenominateurSco']]

df_Sondage_ATM_4 = df_Base_Data_agent_perf_Evita.loc[ (df_Base_Data_agent_perf_Evita['Flux'] == 'Open') | (df_Base_Data_agent_perf_Evita['Flux'] == 'DART') ]
df_Sondage_ATM_4 = df_Sondage_ATM_4[['Date_Format_1', 'CUID','Appels_traites']]

dfg_Sondage_ATM = pd.concat([df_Sondage_ATM_1, df_Sondage_ATM_2, df_Sondage_ATM_3,df_Sondage_ATM_4], join='outer')
t_table_Sondage_ATM = pd.pivot_table(dfg_Sondage_ATM, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
dfg_Sondage_ATM=pd.DataFrame(t_table_Sondage_ATM.to_records())

dfg_Sondage_ATM = pd.merge(left=dfg_Sondage_ATM, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Sondage_ATM = pd.merge(left=dfg_Sondage_ATM, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

#dfg_Sondage_ATM.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Sondage_ATM.csv' , index=False, sep = ';' , decimal=",")

dfg_Sondage_ATM['Datation']=dfg_Sondage_ATM['Datation'].values.astype(str)
dfg_Sondage_ATM['Datation'] = dfg_Sondage_ATM['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-tracage'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Sondage_ATM=Id_dataset+'.Tab_Sondage_ATM'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Sondage_ATM)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Sondage_ATM, table, job_config=job_config)

# **04-Flux**

***Flux***

In [6]:
# @title
from datetime import datetime
df_Flux_1 = df_Base_PrevCapa_Open[['Date_Format_1', 'Flux', 'Sous_Flux' , 'Commande','Capacitaire']]
df_Flux_1['Prevision'] = df_Flux_1['Commande'] * (1/0.9)

df_Flux_2 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART') ]
df_Flux_2 = df_Flux_2[['Date_Format_1' ,'Flux', 'Sous_Flux' , 'Presentes', 'Traites']]
df_Flux_2.loc[df_Flux_2['Flux'] =='DART' , 'Presentes'] = df_Flux_2['Traites']

df_Flux_3 = df_Base_Flux_Quotidien.loc[ (df_Base_Flux_Quotidien['Flux'] == 'Open') | (df_Base_Flux_Quotidien['Flux'] == 'DART') ]
df_Flux_3 = df_Flux_3[['Date_Format_1' ,'Flux', 'Sous_Flux' , 'Dissuades', 'Abandons', 'DMAbd', 'DMR' , 'Traites']]
df_Flux_3['DR'] = df_Flux_3['DMR'] * df_Flux_3['Traites']
df_Flux_3['DAbd'] = df_Flux_3['DMAbd'] * df_Flux_3['Abandons']
df_Flux_3 = df_Flux_3.drop(columns=['Traites' , 'DMAbd', 'DMR'])

df_Flux_4 = df_Base_Data_Skills_Evita.loc[ (df_Base_Data_Skills_Evita['Flux'] == 'Open') | (df_Base_Data_Skills_Evita['Flux'] == 'DART') ]
df_Flux_4 = df_Flux_4[['Date_Format_1' ,'Flux' , 'Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]
df_Flux_4['Sous_Flux'] = 'Mobile'

df_Flux_5 = df_Base_Cubes_Appels.loc[ ((df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART')) & (df_Base_Cubes_Appels['Sous_Flux'] != 'Mobile') ]
df_Flux_5 = df_Flux_5[['Date_Format_1' ,'Flux', 'Sous_Flux' , 'Courts']]

df_Flux_6 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Competence'] == 'cConvEli') | (df_Base_Cubes_Appels['Competence'] == 'mMobEli') ]
df_Flux_6 = df_Flux_6[['Date_Format_1' ,'Flux', 'Sous_Flux' , 'Presentes', 'Traites']]
df_Flux_6.rename(columns = {'Presentes':'Presentes_ELI','Traites':'Traites_ELI'}, inplace = True)

dfg_Flux = pd.concat([df_Flux_1, df_Flux_2, df_Flux_3, df_Flux_4, df_Flux_5, df_Flux_6], join='outer')
t_table_Flux = pd.pivot_table(dfg_Flux , index=['Date_Format_1' , 'Flux', 'Sous_Flux'],aggfunc = 'sum')
dfg_Flux=pd.DataFrame(t_table_Flux.to_records())

dfg_Flux = pd.merge(left=dfg_Flux, right=df_Date_Table, how='left' , on='Date_Format_1')

DN = pd.Timestamp.now().to_period('D')
DN.to_timestamp().strftime("%Y%m%d")
dfg_Flux = dfg_Flux[(dfg_Flux['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

dfg_Flux.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Flux_Jours.csv' , index=False, sep = ';' , decimal=",")

dfg_Flux['Datation']=dfg_Flux['Datation'].values.astype(str)
dfg_Flux['Datation'] = dfg_Flux['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-flux'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Flux=Id_dataset+'.Tab_Flux'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Flux)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Flux, table, job_config=job_config)

df_Data_Prod= dfg_Flux[['Datation' ,'Prevision', 'Presentes' , 'Traites', 'Duree_log_face_au_client_hors_attente' , 'Appels_traites','Capacitaire']]
df_Data_Prod['Flux']= 'Convergents + Mobile'
df_Data_Prod['Activité ou Campagne']= 'ORANGE AC'
t_table_Data_Prod = pd.pivot_table(df_Data_Prod , index=['Datation' , 'Flux', 'Activité ou Campagne'],aggfunc = 'sum')
df_Data_Prod=pd.DataFrame(t_table_Data_Prod.to_records())

df_Data_Prod.loc[(df_Data_Prod['Appels_traites'] > 0) , 'DMT'] = df_Data_Prod['Duree_log_face_au_client_hors_attente'] / df_Data_Prod['Appels_traites']
df_Data_Prod.loc[(df_Data_Prod['Traites'] > 0) , 'QP'] = df_Data_Prod['Presentes'] / df_Data_Prod['Prevision']
df_Data_Prod.loc[(df_Data_Prod['Traites'] > 0) , 'QS'] = df_Data_Prod['Traites'] / df_Data_Prod['Presentes']
df_Data_Prod.loc[(df_Data_Prod['Traites'] > 0) , 'QE'] = df_Data_Prod['Traites'] / df_Data_Prod['Prevision']

df_Data_Prod = df_Data_Prod.drop(columns=['Duree_log_face_au_client_hors_attente', 'Appels_traites'])

df_Data_Prod= df_Data_Prod[['Datation' , 'Flux', 'Activité ou Campagne','Prevision','Capacitaire', 'Presentes' , 'Traites','DMT', 'QP', 'QS', 'QE']]
df_Data_Prod.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_DATA_PROD.csv' , index=False, sep = ';' , decimal=",")



df_Flux_Intervalle_1 = df_Base_PrevCapa_Open_Intervalle[['Date_Format_1', 'Demiheure', 'Flux', 'Sous_Flux' , 'Commande']]
df_Flux_Intervalle_1['Prevision'] = df_Flux_Intervalle_1['Commande'] * (1/0.9)

df_Flux_Intervalle_2 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART') ]
df_Flux_Intervalle_2 = df_Flux_Intervalle_2[['Date_Format_1' , 'Demiheure' ,'Flux', 'Sous_Flux' , 'Presentes', 'Traites']]
df_Flux_Intervalle_2.loc[df_Flux_Intervalle_2['Flux'] =='DART' , 'Presentes'] = df_Flux_2['Traites']

df_Flux_Intervalle_3 = df_Base_Flux_Quotidien.loc[ (df_Base_Flux_Quotidien['Flux'] == 'Open') | (df_Base_Flux_Quotidien['Flux'] == 'DART') ]
df_Flux_Intervalle_3 = df_Flux_Intervalle_3[['Date_Format_1' , 'Demiheure' ,'Flux', 'Sous_Flux' , 'Dissuades', 'Abandons', 'DMAbd', 'DMR' , 'Traites']]
df_Flux_Intervalle_3['DR'] = df_Flux_Intervalle_3['DMR'] * df_Flux_Intervalle_3['Traites']
df_Flux_Intervalle_3['DAbd'] = df_Flux_Intervalle_3['DMAbd'] * df_Flux_Intervalle_3['Abandons']
df_Flux_Intervalle_3 = df_Flux_Intervalle_3.drop(columns=['Traites' , 'DMAbd', 'DMR'])

df_Flux_Intervalle_4 = df_Base_Data_Skills_Evita.loc[ (df_Base_Data_Skills_Evita['Flux'] == 'Open') | (df_Base_Data_Skills_Evita['Flux'] == 'DART') ]
df_Flux_Intervalle_4 = df_Flux_Intervalle_4[['Date_Format_1','Demi_heure' ,'Flux' , 'Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]
df_Flux_Intervalle_4['Sous_Flux'] = 'Mobile'
df_Flux_Intervalle_4.rename(columns = {'Demi_heure':'Demiheure'}, inplace = True)

df_Flux_Intervalle_5 = df_Base_Cubes_Appels.loc[ ((df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART')) & (df_Base_Cubes_Appels['Sous_Flux'] != 'Mobile') ]
df_Flux_Intervalle_5 = df_Flux_Intervalle_5[['Date_Format_1' ,'Demiheure', 'Flux', 'Sous_Flux' , 'Courts']]

df_Flux_Intervalle_6 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Competence'] == 'cConvEli') | (df_Base_Cubes_Appels['Competence'] == 'mMobEli') ]
df_Flux_Intervalle_6 = df_Flux_Intervalle_6[['Date_Format_1' , 'Demiheure' ,'Flux', 'Sous_Flux' , 'Presentes', 'Traites']]
df_Flux_Intervalle_6.rename(columns = {'Presentes':'Presentes_ELI','Traites':'Traites_ELI'}, inplace = True)

dfg_Flux_Intervalle = pd.concat([df_Flux_Intervalle_1, df_Flux_Intervalle_2, df_Flux_Intervalle_3, df_Flux_Intervalle_4, df_Flux_Intervalle_5, df_Flux_Intervalle_6], join='outer')
t_table_Flux_Intervalle = pd.pivot_table(dfg_Flux_Intervalle , index=['Date_Format_1' ,'Demiheure', 'Flux', 'Sous_Flux'],aggfunc = 'sum')
dfg_Flux_Intervalle=pd.DataFrame(t_table_Flux_Intervalle.to_records())

dfg_Flux_Intervalle = pd.merge(left=dfg_Flux_Intervalle, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Flux_Intervalle = pd.merge(left=dfg_Flux_Intervalle, right=df_MT, how='left' , on='Demiheure')

dfg_Flux_Intervalle = dfg_Flux_Intervalle[(dfg_Flux_Intervalle['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

dfg_Flux_Intervalle.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Flux_Jours_Intervalle.csv' , index=False, sep = ';' , decimal=",")

dfg_Flux_Intervalle['Datation']=dfg_Flux_Intervalle['Datation'].values.astype(str)
dfg_Flux_Intervalle['Datation'] = dfg_Flux_Intervalle['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-flux'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Flux_Intervalle=Id_dataset+'.Tab_Flux_Intervalle'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Flux_Intervalle)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Flux_Intervalle, table, job_config=job_config)




df_Comp_1 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART') ]
df_Comp_1 = df_Comp_1[['Date_Format_1' ,'Flux', 'Competence' ,'Demiheure', 'Presentes', 'Traites']]
df_Comp_1.loc[df_Comp_1['Flux'] =='DART' , 'Presentes'] = df_Comp_1['Traites']
df_Comp_1.loc[df_Comp_1['Competence'] =='DcConvAC' , 'Competence'] = 'cConvAC'

df_Comp_2 = df_Base_Flux_Quotidien.loc[ (df_Base_Flux_Quotidien['Flux'] == 'Open') | (df_Base_Flux_Quotidien['Flux'] == 'DART') ]
df_Comp_2 = df_Comp_2[['Date_Format_1' ,'Flux', 'Competence' , 'Demiheure', 'Dissuades', 'Abandons', 'DMAbd', 'DMR' , 'Traites']]
df_Comp_2['DR'] = df_Comp_2['DMR'] * df_Comp_2['Traites']
df_Comp_2['DAbd'] = df_Comp_2['DMAbd'] * df_Comp_2['Abandons']
df_Comp_2 = df_Comp_2.drop(columns=['Traites' , 'DMAbd', 'DMR'])
df_Comp_2.loc[df_Comp_2['Competence'] =='DcConvAC' , 'Competence'] = 'cConvAC'

df_Comp_3 = df_Base_Data_Skills_Evita.loc[ (df_Base_Data_Skills_Evita['Flux'] == 'Open') | (df_Base_Data_Skills_Evita['Flux'] == 'DART') ]
df_Comp_3 = df_Comp_3[['Date_Format_1' ,'Flux' , 'Competence','Demi_heure', 'Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC']]
df_Comp_3.rename(columns = {'Demi_heure':'Demiheure'}, inplace = True)
df_Comp_3.loc[df_Comp_3['Competence'] =='DcConvAC' , 'Competence'] = 'cConvAC'

df_Comp_4 = df_Base_Cubes_Appels.loc[ ((df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART')) & (df_Base_Cubes_Appels['Sous_Flux'] != 'Mobile') ]
df_Comp_4 = df_Comp_4[['Date_Format_1' ,'Flux', 'Competence' , 'Demiheure', 'Courts']]
df_Comp_4.loc[df_Comp_4['Competence'] =='DcConvAC' , 'Competence'] = 'cConvAC'

dfg_Comp = pd.concat([df_Comp_1, df_Comp_2, df_Comp_3, df_Comp_4], join='outer')
t_table_Comp = pd.pivot_table(dfg_Comp , index=['Date_Format_1' , 'Flux', 'Competence', 'Demiheure'],aggfunc = 'sum')
dfg_Comp=pd.DataFrame(t_table_Comp.to_records())

dfg_Comp = pd.merge(left=dfg_Comp, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Comp = pd.merge(left=dfg_Comp, right=df_MT, how='left' , on='Demiheure')

DN = pd.Timestamp.now().to_period('D')
DN.to_timestamp().strftime("%Y%m%d")
dfg_Comp = dfg_Comp[(dfg_Comp['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

#dfg_Flux.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Flux_Jours.csv' , index=False, sep = ';' , decimal=",")

dfg_Comp['Datation']=dfg_Comp['Datation'].values.astype(str)
dfg_Comp['Datation'] = dfg_Comp['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-flux'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Flux_Comp=Id_dataset+'.Tab_Flux_Comp'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Flux_Comp)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Comp, table, job_config=job_config)

# **04-Commerce**

***Commerce Global***

In [7]:
# @title
df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])
df_1 = df_Base_Commerce.drop(columns=['Lib_Equipe', 'KPI','SousKpi','KPI_Item','Datation'])
df_2 = df_Base_RT.drop(columns=['Lib_Equipe', 'FluxArrivee','Offre', 'Univers', 'MetierTraitant','exploitables', 'Reit1J', 'Reit7J', 'Appels_1min', 'NbreTransfertsAccompAveug','Datation'])
df_3 = df_Base_Commerce.drop(columns=['Lib_Equipe', 'FamilleKpiHisto','SousKpi', 'KPI','Datation'])
df_4 = df_Base_CmdFibre.drop(columns=['Lib_Equipe','Datation'])

df_Ext_1 = df_Base_Commerce[['CUID' , 'KPI_Item' , 'FamilleKpiHisto' , 'Date_Format_1' , 'Flux' , 'Total']]
df_Ext_2 = df_Base_Commerce_Annulation.loc[ (df_Base_Commerce_Annulation['FamilleKpiHisto'] != 'OPEN') & (df_Base_Commerce_Annulation['FamilleKpiHisto'] != 'FORFAITS MOBILE ORANGE CLIENTS OPEN ML') ]
df_Ext_2 = df_Ext_2[['CUID' , 'KPI_Item' , 'FamilleKpiHisto' , 'Date_Format_1' , 'Flux' , 'DEN' , 'NUM' , 'Annule']]

df_1['FamilleKpiHisto'] = df_1['FamilleKpiHisto'].apply(lambda x: x.replace(' ', '_'))
df_1['FamilleKpiHisto'] = df_1['FamilleKpiHisto'].apply(lambda x: x.replace('(DONT_DOM)', 'DOM'))

pivoted=pd.pivot_table(df_1, values='Total', index =['Date_Format_1','CUID','Flux'] ,columns=['FamilleKpiHisto'], aggfunc='sum')
df_1=pd.DataFrame(pivoted.to_records())

pivoted=pd.pivot_table(df_2, values='NbreAppelTraite', index =['Date_Format_1','CUID','Flux'] ,columns=['Sous_Flux'], aggfunc='sum')
df_2=pd.DataFrame(pivoted.to_records())


pivoted=pd.pivot_table(df_3, values='Total', index =['Date_Format_1','CUID','Flux'] ,columns=['KPI_Item'], aggfunc='sum')
df_3=pd.DataFrame(pivoted.to_records())

df_5 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART') ]
df_5 = df_5[['Date_Format_1' ,'Flux', 'Sous_Flux', 'Traites']]
pivoted=pd.pivot_table(df_5, values='Traites', index =['Date_Format_1','Flux'] ,columns=['Sous_Flux'], aggfunc='sum')
df_5=pd.DataFrame(pivoted.to_records())
df_5.rename(columns = {'ADSL':'G_ADSL','Convergent':'G_Convergent','Fibre':'G_Fibre','Mobile':'G_Mobile','Premium':'G_Premium'}, inplace = True)
df_5['CUID']='Global'

#display(df_5)

dfg = pd.concat([df_1, df_2, df_3, df_4,df_5], join='outer')
t_table = pd.pivot_table(dfg, index=['Date_Format_1','CUID', 'Flux'],aggfunc = 'sum')
dfg=pd.DataFrame(t_table.to_records())

df_Ext = pd.concat([df_Ext_1, df_Ext_2], join='outer')
t_table = pd.pivot_table(df_Ext, index=['CUID' , 'KPI_Item' , 'FamilleKpiHisto' , 'Date_Format_1' , 'Flux'],aggfunc = 'sum')
df_Ext=pd.DataFrame(t_table.to_records())

dfg = pd.merge(left=dfg, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg = pd.merge(left=dfg, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])
df_Ext = pd.merge(left=df_Ext, right=df_Date_Table, how='left' , on='Date_Format_1')
df_Ext = pd.merge(left=df_Ext, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

dfg.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Commerce.csv' , index=False, sep = ';' , decimal=",")

dfg['Datation']=dfg['Datation'].values.astype(str)
dfg['Datation'] = dfg['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

df_Ext['Datation']=df_Ext['Datation'].values.astype(str)
df_Ext['Datation'] = df_Ext['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))


Id_Project='orange-open-commerce'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Commerce=Id_dataset+'.Tab_Commerce'
Id_table_Commerce_Ext=Id_dataset+'.Tab_Commerce_Ext'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Commerce)
bq.delete_table(Id_table_Commerce, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg, Id_table_Commerce, job_config=job_config)

table = bigquery.Table(Id_table_Commerce_Ext)
bq.delete_table(Id_table_Commerce_Ext, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_Ext, Id_table_Commerce_Ext, job_config=job_config)

# **05-Réitération**

***Réitération Global***

In [8]:
# @title
df_1 = df_Base_Reit.drop(columns=['Lib_Equipe', 'TypeActivite','Offre', 'Tendance'])
df_2 = df_Base_RT.drop(columns=['Date_Format_1', 'Lib_Equipe', 'FluxArrivee','Offre', 'Univers', 'MetierTraitant','exploitables', 'Reit1J', 'Reit7J', 'Appels_1min'])

dfg = pd.concat([df_1, df_2], join='outer')
t_table = pd.pivot_table(dfg, index=['Datation','CUID', 'Flux', 'Sous_Flux'],aggfunc = 'sum')
dfg=pd.DataFrame(t_table.to_records())

dfg = pd.merge(left=dfg, right=df_Effectif_Global, how='left' , on=['Datation','CUID'])
dfg['Datation']=dfg['Datation'].values.astype(str)
dfg['Datation'] = dfg['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
dfg.loc[(dfg['Datation'] <= datetime.now() - timedelta(days=8)), 'exploitables_7'] = dfg['exploitables']
dfg.loc[(dfg['Datation'] <= datetime.now() - timedelta(days=8)), 'Reit7J_7'] = dfg['Reit7J']

#dfg.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Reiteration_Global.csv' , index=False, sep = ';' , decimal=",")

Id_Project='orange-open-reiteration'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Reit=Id_dataset+'.Tab_Reiteration'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Reit)
bq.delete_table(Id_table_Reit, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg, Id_table_Reit, job_config=job_config)

***Réitération ND***

In [9]:
# @title
dfg_Reiteration_ND = df_Base_Reit_ND[['Date_Format_1', 'CUID', 'NumeroDossier', 'exploitables', 'Reit1h', 'Reit1J', 'Reit7J']]

dfg_Reiteration_ND = pd.merge(left=dfg_Reiteration_ND, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Reiteration_ND = pd.merge(left=dfg_Reiteration_ND, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

dfg_Reiteration_ND['Datation']=dfg_Reiteration_ND['Datation'].values.astype(str)
dfg_Reiteration_ND['Datation'] = dfg_Reiteration_ND['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-reiteration'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Reit_ND=Id_dataset+'.Tab_Reiteration_ND'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Reit_ND)
bq.delete_table(Id_table_Reit_ND, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Reiteration_ND, Id_table_Reit_ND, job_config=job_config)

# **06-Production**

***Open : Production Global***

In [10]:
# @title
df_Effectif = df_Effectif_Global[['Date_Format_1','CUID','Matricule']]
df_1 = df_Base_Data_agent_Retrait_Evita.drop(columns=['Datation'])
df_1 = pd.merge(left=df_1, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])
df_1.dropna(subset=['Matricule'], inplace=True)
df_1 = df_1.drop(columns=['Matricule'])

df_2 = df_Base_Data_agent_perf_Evita.loc[ (df_Base_Data_agent_perf_Evita['Flux'] == 'Open') | (df_Base_Data_agent_perf_Evita['Flux'] == 'DART') ]
df_2 = df_2.drop(columns=['Datation','Competence' ,'Flux'])

#df_2.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Production.csv' , index=False, sep = ';' , decimal=",")

df_Effectif = df_Effectif_Global[['Date_Format_1','CUID','Log_PRA']]
df_3 = df_Base_PRA.loc[ (df_Base_PRA['Flux'] == 'OPEN') | (df_Base_PRA['Flux'] == 'DART') ]
df_3 = df_3.drop(columns=['Datation' ,'NOM','PRENOM','AFFECTATION','Flux'])
df_3 = pd.merge(left=df_3, right=df_Effectif, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_3 = df_3.drop(columns=['Log_PRA' ,'MATRICULE'])
df_3.loc[df_3['CUID'].isnull(), 'CUID'] = 'Non Defini'

pivoted=pd.pivot_table(df_3, values='H_REA', index =['Date_Format_1'] , aggfunc='sum')
df_PRA_Dates=pd.DataFrame(pivoted.to_records())
df_PRA_Dates= df_PRA_Dates[df_PRA_Dates['H_REA'] == 0]
df_PRA_Dates=df_PRA_Dates["Date_Format_1"].unique()

df_4 = df_3.copy()
df_4.rename(columns={'H_PLA': 'H_PLA_ABS','H_REA': 'H_REA_ABS','H_ABS': 'H_ABS_ABS'}, inplace=True)
df_4 = df_4[~df_4['Date_Format_1'].isin(df_PRA_Dates)]

df_5 = df_Base_REAL.loc[ (df_Base_REAL['Flux'] == 'OPEN') | (df_Base_REAL['Flux'] == 'DART') ]
df_5 = df_5.loc[ (df_5['Activite'] == 'Improd')]
df_5 = df_5.drop(columns=['Datation' ,'NOM','PRENOM','AFFECTATION','ACTIVITE_LIB','TELETRAVAIL','Flux','Activite'])
df_5 = pd.merge(left=df_5, right=df_Effectif, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_5 = df_5.drop(columns=['Log_PRA' ,'MATRICULE'])

dfg_Production = pd.concat([df_1, df_2, df_3, df_4, df_5], join='outer')
t_table = pd.pivot_table(dfg_Production, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
dfg_Production=pd.DataFrame(t_table.to_records())

df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])
dfg_Production = pd.merge(left=dfg_Production, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Production = pd.merge(left=dfg_Production, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])



dfg_Production['Datation']=dfg_Production['Datation'].values.astype(str)
dfg_Production['Datation'] = dfg_Production['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-production'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Production=Id_dataset+'.Tab_Production'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Production)
bq.delete_table(Id_table_Production, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Production, Id_table_Production, job_config=job_config)

***Open : Synthèse Perf Agent***

In [11]:
# @title
df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])

df_Synthese_Perf_1 = df_Base_Data_agent_perf_Evita.loc[ (df_Base_Data_agent_perf_Evita['Flux'] == 'Open') | (df_Base_Data_agent_perf_Evita['Flux'] == 'DART') ]
df_Synthese_Perf_1 = df_Synthese_Perf_1[['CUID' , 'Appels_traites' , 'Appels_courts' , 'Appels_abandon_sur_sonnerie' , 'Appels_rotation_sur_sonnerie' , 'Appels_mis_en_attente' , 'Duree_log_total' , 'Duree_log_face_au_client' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_traitement' , 'Duree_conversation_appels_entrants' , 'Duree_garde_appels_entrants' , 'Duree_pas_libre' , 'Duree_wrapup' , 'Duree_pause_entre_appels' , 'Duree_THC' , 'Date_Format_1']]

df_Synthese_Perf_2 = df_Base_PRA.loc[ (df_Base_PRA['Flux'] == 'OPEN') | (df_Base_PRA['Flux'] == 'DART')  ]
df_Synthese_Perf_2 = df_Synthese_Perf_2[['MATRICULE','H_PLA' , 'H_REA' , 'H_ABS' , 'Date_Format_1']]
df_Synthese_Perf_2 = pd.merge(left=df_Synthese_Perf_2, right=df_Effectif, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_Synthese_Perf_2 = df_Synthese_Perf_2[['CUID','H_PLA' , 'H_REA' , 'H_ABS' , 'Date_Format_1']]
df_Synthese_Perf_2.loc[df_Synthese_Perf_2['CUID'].isnull(), 'CUID'] = 'Non Defini'

t_table_Synthese_Perf_PRA = pd.pivot_table(df_Synthese_Perf_2, index=['Date_Format_1'],aggfunc = 'sum')
dfg_Synthese_Perf_PRA=pd.DataFrame(t_table_Synthese_Perf_PRA.to_records())
dfg_Synthese_Perf_PRA = dfg_Synthese_Perf_PRA.loc[dfg_Synthese_Perf_PRA['H_REA'] > 0]
Dates_Perf_PRA=dfg_Synthese_Perf_PRA["Date_Format_1"].unique()
df_Synthese_Perf_6 = df_Synthese_Perf_2[df_Synthese_Perf_2['Date_Format_1'].isin(Dates_Perf_PRA)]
df_Synthese_Perf_6.rename(columns={'H_PLA': 'H_PLA_ABS','H_REA': 'H_REA_ABS','H_ABS': 'H_ABS_ABS'}, inplace=True)

df_Synthese_Perf_3 = df_Base_REAL.loc[ ((df_Base_REAL['Flux'] == 'OPEN') | (df_Base_REAL['Flux'] == 'DART')) & (df_Base_REAL['Activite'] == 'Improd')  ]
df_Synthese_Perf_3 = df_Synthese_Perf_3[['MATRICULE','DUREE', 'Date_Format_1']]
df_Synthese_Perf_3 = pd.merge(left=df_Synthese_Perf_3, right=df_Effectif, how='left' , left_on=['Date_Format_1','MATRICULE'], right_on=['Date_Format_1','Log_PRA'])
df_Synthese_Perf_3 = df_Synthese_Perf_3[['CUID','DUREE', 'Date_Format_1']]

df_Synthese_Perf_4 = df_Base_Data_agent_Retrait_Evita[['CUID' , 'Total_Logge' , 'Duree_retrait_autres' , 'Duree_retrait_formation' , 'Duree_retrait_management' , 'Duree_retrait_pause' , 'Date_Format_1']]
df_Synthese_Perf_4 = pd.merge(left=df_Synthese_Perf_4, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])
df_Synthese_Perf_4.dropna(subset=['Matricule'], inplace=True)
df_Synthese_Perf_4 = df_Synthese_Perf_4[['CUID' , 'Total_Logge' , 'Duree_retrait_autres' , 'Duree_retrait_formation' , 'Duree_retrait_management' , 'Duree_retrait_pause' , 'Date_Format_1']]

df_Synthese_Perf_5 = df_Base_AvisClient[['Date_Format_1', 'intervenant_code' ,'Etoile_1' , 'Etoile_2' , 'Etoile_3' , 'Etoile_4' , 'Etoile_5' , 'Satisfaction_Attente' , 'Satisfaction_Acceuil' , 'Satisfaction_Ecoute' , 'Satisfaction_Solution' , 'Satisfaction_Découverte' , 'Den_Satisfaction_Découverte' , 'TEP' , 'Den_TEP' , 'Rappel' , 'Traitement' , 'Main_Equipe' , 'Non_Traite']]
df_Synthese_Perf_5.rename(columns = {'intervenant_code':'CUID','Den_Satisfaction_Découverte':'Den_Satisfaction_Decouverte','Satisfaction_Découverte':'Satisfaction_Decouverte'}, inplace = True)

df_Synthese_Perf_7 = df_Base_Reit[['Datation', 'CUID' , 'exploitables' , 'Appels_1min' , 'Reit1h' , 'Reit1J' , 'Reit7J' , 'Date_Format_1']]
df_Synthese_Perf_7['Datation']=df_Synthese_Perf_7['Datation'].values.astype(str)
df_Synthese_Perf_7['Datation'] = df_Synthese_Perf_7['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
df_Synthese_Perf_7.loc[(df_Synthese_Perf_7['Datation'] <= datetime.now() - timedelta(days=8)), 'exploitables_7'] = df_Synthese_Perf_7['exploitables']
df_Synthese_Perf_7.loc[(df_Synthese_Perf_7['Datation'] <= datetime.now() - timedelta(days=8)), 'Reit7J_7'] = df_Synthese_Perf_7['Reit7J']
df_Synthese_Perf_7 = df_Synthese_Perf_7.drop(columns=['Datation'])

df_Synthese_Perf_8 = df_Base_RT[['CUID' , 'NbreAppelTraite' , 'NbreTransfertsAccompAveug' , 'Date_Format_1']]

df_Synthese_Perf_9 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp','N_dossier']]
df_Synthese_Perf_9['Nbre_Tracage'] = 1
df_Synthese_Perf_9.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID' , 'N_dossier':'NumeroDossier'}, inplace = True)
df_Synthese_Perf_9 = pd.concat([df_Synthese_Perf_9, df_Base_Reit_ND], join='outer')
df_Synthese_Perf_9 = df_Synthese_Perf_9.drop(columns=['Datation','Reit1J','Reit1h','Reit7J'])
t_table_Synthese_Perf_9 = pd.pivot_table(df_Synthese_Perf_9, index=['CUID', 'Date_Format_1','NumeroDossier'],aggfunc = 'sum')
df_Synthese_Perf_9=pd.DataFrame(t_table_Synthese_Perf_9.to_records())
df_Synthese_Perf_9.loc[df_Synthese_Perf_9['Nbre_Tracage'] >= df_Synthese_Perf_9['exploitables'] , 'Nbre_Tracage_ND'] = df_Synthese_Perf_9['exploitables']
df_Synthese_Perf_9.loc[df_Synthese_Perf_9['Nbre_Tracage'] < df_Synthese_Perf_9['exploitables'] , 'Nbre_Tracage_ND'] = df_Synthese_Perf_9['Nbre_Tracage']
df_Synthese_Perf_9 = df_Synthese_Perf_9.drop(columns=['NumeroDossier'])
t_table_Synthese_Perf_9 = pd.pivot_table(df_Synthese_Perf_9, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
df_Synthese_Perf_9=pd.DataFrame(t_table_Synthese_Perf_9.to_records())

df_Synthese_Perf_9.loc[df_Synthese_Perf_9['Date_Format_1'] <= df_Base_Chorus['Date_Format_1'].max() , 'exploitables_Tracage'] = df_Synthese_Perf_9['exploitables']
df_Synthese_Perf_9 = df_Synthese_Perf_9.drop(columns=['exploitables'])

df_Synthese_Perf_10 = df_Base_Commerce.drop(columns=['Lib_Equipe', 'KPI','SousKpi','KPI_Item','Datation'])
df_Synthese_Perf_10['FamilleKpiHisto'] = df_Synthese_Perf_10['FamilleKpiHisto'].apply(lambda x: x.replace(' ', '_'))
df_Synthese_Perf_10['FamilleKpiHisto'] = df_Synthese_Perf_10['FamilleKpiHisto'].apply(lambda x: x.replace('(DONT_DOM)', 'DOM'))
pivoted_Synthese_Perf_10=pd.pivot_table(df_Synthese_Perf_10, values='Total', index =['Date_Format_1','CUID','Flux'] ,columns=['FamilleKpiHisto'], aggfunc='sum')
df_Synthese_Perf_10=pd.DataFrame(pivoted_Synthese_Perf_10.to_records())

df_Synthese_Perf_11 = df_Base_Commerce[['Date_Format_1', 'CUID','Total']]
df_Synthese_Perf_11.rename(columns = {'Total':'Total_Commerce'}, inplace = True)

pivoted_Synthese_Perf_12=pd.pivot_table(df_Base_RT, values='NbreAppelTraite', index =['Date_Format_1','CUID','Flux'] ,columns=['Sous_Flux'], aggfunc='sum')
df_Synthese_Perf_12=pd.DataFrame(pivoted_Synthese_Perf_12.to_records())

df_Synthese_Perf_13 = df_Base_Commerce.drop(columns=['Lib_Equipe', 'FamilleKpiHisto','SousKpi', 'KPI','Datation'])
pivoted_Synthese_Perf_13=pd.pivot_table(df_Synthese_Perf_13, values='Total', index =['Date_Format_1','CUID','Flux'] ,columns=['KPI_Item'], aggfunc='sum')
df_Synthese_Perf_13=pd.DataFrame(pivoted_Synthese_Perf_13.to_records())

df_Synthese_Perf_14 = df_Base_HAPPY[['Date_Format_1', 'CUID','Happy','Chorus','Hagrid']]

df_Synthese_Perf_15 = df_Base_DiagonalMobile[['Date_Format_1', 'CUID','NbCas','NbScript']]

df_Synthese_Perf_16 = df_Base_Reclamation[['Date_Format_1', 'CUID','Total_Initialisees_au_N1','Total_Transferees_au_N2']]
df_Synthese_Perf_16 = df_Synthese_Perf_16[df_Synthese_Perf_16['Date_Format_1'] < df_Synthese_Perf_16['Date_Format_1'].max()-1]

df_Synthese_Perf_17 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp','N_client']]
df_Synthese_Perf_17.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Synthese_Perf_17 = df_Synthese_Perf_17.drop_duplicates(subset=['Date_Format_1', 'N_client','CUID'] , keep="first", inplace=False)
df_Synthese_Perf_17['Nbre_Tracage_NClient_Date'] = 1
df_Synthese_Perf_17 = df_Synthese_Perf_17.drop(columns=['N_client'])

df_Synthese_Perf_18 = df_Base_Chorus[['Date_Format_1', 'Code_Alliance_Conseiller_Resp','N_client']]
df_Synthese_Perf_18.rename(columns = {'Code_Alliance_Conseiller_Resp':'CUID'}, inplace = True)
df_Synthese_Perf_18['Mois']=df_Synthese_Perf_18['Date_Format_1'].values.astype(str)
df_Synthese_Perf_18['Mois'] = df_Synthese_Perf_18['Mois'].str[:6]
df_Synthese_Perf_18 = df_Synthese_Perf_18.drop_duplicates(subset=['Mois', 'N_client','CUID'] , keep="first", inplace=False)
df_Synthese_Perf_18['Nbre_Tracage_NClient_Mois'] = 1
df_Synthese_Perf_18 = df_Synthese_Perf_18.drop(columns=['N_client','Mois'])

df_Synthese_Perf_1.loc[df_Synthese_Perf_1['Date_Format_1'] <= df_Synthese_Perf_18['Date_Format_1'].max() , 'Appels_traites_Tracage'] = df_Synthese_Perf_1['Appels_traites']

dfg_Synthese_Perf = pd.concat([df_Synthese_Perf_1, df_Synthese_Perf_2, df_Synthese_Perf_3, df_Synthese_Perf_4, df_Synthese_Perf_5, df_Synthese_Perf_6, df_Synthese_Perf_7, df_Synthese_Perf_8, df_Synthese_Perf_9, df_Synthese_Perf_10, df_Synthese_Perf_11, df_Synthese_Perf_12,df_Synthese_Perf_13,df_Synthese_Perf_14,df_Synthese_Perf_15,df_Synthese_Perf_16,df_Synthese_Perf_17,df_Synthese_Perf_18], join='outer')
t_table_Synthese_Perf = pd.pivot_table(dfg_Synthese_Perf, index=['CUID', 'Date_Format_1'],aggfunc = 'sum')
dfg_Synthese_Perf=pd.DataFrame(t_table_Synthese_Perf.to_records())

dfg_Synthese_Perf = pd.merge(left=dfg_Synthese_Perf, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg_Synthese_Perf = pd.merge(left=dfg_Synthese_Perf, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

dfg_Synthese_Perf.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Prime.csv' , index=False, sep = ';' , decimal=",")

dfg_Synthese_Perf['Datation']=dfg_Synthese_Perf['Datation'].values.astype(str)
dfg_Synthese_Perf['Datation'] = dfg_Synthese_Perf['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

Id_Project='orange-open-production'
Id_dataset=Id_Project+'.Orange_Open'
Id_table_Synthese_Perf=Id_dataset+'.Tab_Synthese_Perf_Agent'
bq = bigquery.Client(project=Id_Project)

table = bigquery.Table(Id_table_Synthese_Perf)
bq.delete_table(table, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg_Synthese_Perf, table, job_config=job_config)

DNN = pd.Timestamp.now().to_period('D') - 40
DNN.to_timestamp().strftime("%Y%m%d")

DNN1 = pd.Timestamp.now().to_period('D')

df_DATA = dfg_Synthese_Perf[['Date_Format_1', 'Datation' , 'Log_PRA', 'Equipe', 'CDP', 'Localiastion', 'Affectation', 'Appels_traites' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_log_total' , 'Duree_retrait_formation' , 'Duree_retrait_management' , 'Duree_retrait_pause' , 'Etoile_1' , 'Etoile_2' , 'Etoile_3' , 'Etoile_4' , 'Etoile_5' , 'H_REA' , 'Reit1J' , 'Reit1h' , 'Reit7J' , 'exploitables', 'NbreTransfertsAccompAveug', 'Total_Commerce']]
df_DATA['Connecte'] =  df_DATA['Duree_log_total'] /3600
df_DATA['Connecte2'] =  df_DATA['Duree_log_total'] /3600
df_DATA['Formation'] =  df_DATA['Duree_retrait_formation'] /3600
df_DATA['Manager'] =  df_DATA['Duree_retrait_management'] /3600
df_DATA['MYT'] =  df_DATA['Duree_retrait_management'] /3600
df_DATA['Dispo'] =  df_DATA['Duree_dispo_CC'] /3600
df_DATA['NUMQlt'] =  df_DATA['Etoile_5'] - df_DATA['Etoile_3']  - df_DATA['Etoile_2']  - df_DATA['Etoile_1']
df_DATA['DENNUMQlt'] =  df_DATA['Etoile_1'] + df_DATA['Etoile_2']  + df_DATA['Etoile_3'] + df_DATA['Etoile_4'] + df_DATA['Etoile_5']
df_DATA['Duree_Traitement'] =  df_DATA['Duree_log_face_au_client_hors_attente'] /3600
df_DATA['CA'] =  df_DATA['Appels_traites'] * 3.09
df_DATA.loc[df_DATA['Localiastion'] == 'Télétravail' , 'Localiastion'] = 'TT'
df_DATA['Pause'] =df_DATA['Connecte'] -  df_DATA['Duree_Traitement']  - df_DATA['Dispo'] - df_DATA['Manager'] - df_DATA['Formation']
df_DATA['Pause2'] =df_DATA['Duree_retrait_pause'] /3600
df_DATA['Heures_Prod'] =df_DATA['Connecte'] - df_DATA['Dispo'] - df_DATA['Manager'] - df_DATA['Formation'] - df_DATA['Pause']
df_DATA['Activite2'] ='PAL_ORANGE OPEN'
df_DATA['Activite'] ='PAL_ORANGE OPEN'
df_DATA['Activite3'] ='ORANGE_AC_OPEN'

#DATA_AGENT-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_DATA_AGENT = df_DATA[['Date_Format_1','Datation' , 'Log_PRA', 'Equipe', 'CDP', 'Localiastion', 'Activite3', 'Appels_traites','Duree_Traitement','NUMQlt', 'DENNUMQlt' , 'Connecte', 'Formation', 'Manager', 'Pause', 'Dispo', 'MYT', 'CA']]
df_DATA_AGENT.sort_values(by=['Datation'], inplace=True, ascending=True)
df_DATA_AGENT.dropna(subset=['Log_PRA'], inplace=True)
df_DATA_AGENT = df_DATA_AGENT[(df_DATA_AGENT['Date_Format_1'] > int(DNN.to_timestamp().strftime("%Y%m%d")))]
df_DATA_AGENT = df_DATA_AGENT.drop(columns=['Date_Format_1'])

df_DATA_AGENT.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_DATA_AGENT.csv' , index=False, sep = ';' , decimal=",")

#ScoreCard-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
df_DATA_ScoreCard_1 = df_DATA[['Date_Format_1', 'Log_PRA', 'Activite', 'Activite2', 'Connecte', 'Pause2', 'Manager', 'Formation', 'Dispo', 'CA', 'Appels_traites', 'Connecte2', 'Heures_Prod', 'H_REA', 'NbreTransfertsAccompAveug', 'Etoile_1' , 'Etoile_2' , 'Etoile_3' , 'Etoile_4' , 'Etoile_5', 'Reit7J' , 'Reit1h', 'exploitables','Duree_log_face_au_client_hors_attente','Total_Commerce' ]]
df_DATA_ScoreCard_1.dropna(subset=['Log_PRA'], inplace=True)
df_DATA_ScoreCard_1 = df_DATA_ScoreCard_1[(df_DATA_ScoreCard_1['Date_Format_1'] > int(DNN.to_timestamp().strftime("%Y%m%d")))]
df_DATA_ScoreCard_1.rename(columns = {'Log_PRA':'Matricule', 'Activite':'Activite', 'Activite2':'Campagne', 'Connecte':'Heures_Connecte', 'Pause2':'Heures_Pause', 'Manager':'Heures_Brief', 'Formation':'Heures_FC', 'Dispo':'Heures_Dispo', 'CA':'Chiffre_Affaire', 'Appels_traites':'Appels_traitees', 'Connecte2':'Connexion', 'Heures_Prod':'Heures_Productives', 'H_REA':'Heures_Realisees', 'NbreTransfertsAccompAveug':'Appels_Transfert', 'Etoile_1':'NB_1Etoile', 'Etoile_2':'NB_2Etoile', 'Etoile_3':'NB_3Etoile', 'Etoile_4':'NB_4Etoile', 'Etoile_5':'NB_5Etoile', 'Reit7J':'Appels_reiit_7j', 'Reit1h':'Appels_reiit_1h', 'exploitables':'Appels_exploitable', 'Duree_log_face_au_client_hors_attente':'Duree_traitement', 'Total_Commerce':'Ventes'}, inplace = True)

df_DATA_ScoreCard_2 = df_DATA_ScoreCard_1.drop(columns=['Matricule'])
df_DATA_ScoreCard_2['Matricule'] =  'Global'

df_DATA_ScoreCard_3 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART') ]
df_DATA_ScoreCard_3 = df_DATA_ScoreCard_3[['Date_Format_1' , 'Presentes']]
df_DATA_ScoreCard_3['Matricule'] =  'Global'
df_DATA_ScoreCard_3['Activite'] =  'PAL_ORANGE OPEN'
df_DATA_ScoreCard_3['Campagne'] =  'PAL_ORANGE OPEN'
df_DATA_ScoreCard_3 = df_DATA_ScoreCard_3[(df_DATA_ScoreCard_3['Date_Format_1'] > int(DNN.to_timestamp().strftime("%Y%m%d")))]

df_DATA_ScoreCard = pd.concat([df_DATA_ScoreCard_3, df_DATA_ScoreCard_2], join='outer')
t_table_DATA_ScoreCard = pd.pivot_table(df_DATA_ScoreCard , index=['Date_Format_1','Matricule','Activite','Campagne'],aggfunc = 'sum')
df_DATA_ScoreCard=pd.DataFrame(t_table_DATA_ScoreCard.to_records())

df_DATA_ScoreCard = pd.concat([df_DATA_ScoreCard, df_DATA_ScoreCard_1], join='outer')
t_table_DATA_ScoreCard = pd.pivot_table(df_DATA_ScoreCard , index=['Date_Format_1','Matricule','Activite','Campagne'],aggfunc = 'sum')
df_DATA_ScoreCard=pd.DataFrame(t_table_DATA_ScoreCard.to_records())

df_Date_Table_ScoreCard = df_Date_Table[['Date_Format_1', 'Datation']]
df_DATA_ScoreCard = pd.merge(left=df_DATA_ScoreCard, right=df_Date_Table_ScoreCard, how='left' , on='Date_Format_1')
df_DATA_ScoreCard = df_DATA_ScoreCard.drop(columns=['Date_Format_1'])
df_DATA_ScoreCard.rename(columns = {'Datation':'Date','Presentes':'Appels_Recu'}, inplace = True)

df_DATA_ScoreCard = df_DATA_ScoreCard[['Date' , 'Matricule' , 'Activite' , 'Campagne' , 'Heures_Connecte' , 'Heures_Pause' , 'Heures_Brief' , 'Heures_FC' , 'Heures_Dispo' , 'Chiffre_Affaire' , 'Appels_traitees' , 'Connexion' , 'Heures_Productives' , 'Heures_Realisees' , 'Appels_Transfert' , 'NB_1Etoile' , 'NB_2Etoile' , 'NB_3Etoile' , 'NB_4Etoile' , 'NB_5Etoile' , 'Appels_reiit_7j' , 'Appels_reiit_1h' , 'Appels_exploitable' , 'Duree_traitement' , 'Ventes' , 'Appels_Recu']]

df_DATA_ScoreCard.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/orange_open_'+DNN1.to_timestamp().strftime("%d%m%Y")+'.csv' , index=False, sep = ';' , decimal=",")

#Flash KPI's-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_DATA_Global_1 = dfg_Synthese_Perf[['Date_Format_1' , 'Appels_traites' , 'Duree_dispo_CC' , 'Duree_log_face_au_client_hors_attente' , 'Duree_log_total' , 'Duree_retrait_formation' , 'Duree_retrait_management' , 'Duree_retrait_pause' , 'Etoile_1' , 'Etoile_2' , 'Etoile_3' , 'Etoile_4' , 'Etoile_5' , 'Den_Satisfaction_Decouverte' , 'Satisfaction_Decouverte' , 'H_REA_ABS' , 'H_PLA_ABS' , 'H_ABS_ABS' , 'DUREE' , 'NbreAppelTraite' , 'NbreTransfertsAccompAveug' , 'Reit1J' , 'Reit1h' , 'Reit7J' , 'Reit7J_7' , 'exploitables' , 'exploitables_7' , 'exploitables_Tracage' , 'Nbre_Tracage_ND' ,'Happy','Chorus','Hagrid','NbCas','NbScript']]

df_DATA_Global_2 = df_Base_Cubes_Appels.loc[ (df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART') ]
df_DATA_Global_2.loc[df_DATA_Global_2['Flux'] =='DART' , 'Presentes'] = df_DATA_Global_2['Traites']
df_DATA_Global_2 = df_DATA_Global_2[['Date_Format_1' , 'Presentes', 'Traites']]

df_DATA_Global_3 = df_Base_PrevCapa_Open[['Date_Format_1', 'Commande','Capacitaire']]
df_DATA_Global_3['Prevision'] = df_DATA_Global_3['Commande'] * (1/0.9)

df_DATA_Global_4 = df_Base_Cubes_Appels.loc[ ((df_Base_Cubes_Appels['Flux'] == 'Open') | (df_Base_Cubes_Appels['Flux'] == 'DART')) & (df_Base_Cubes_Appels['Sous_Flux'] != 'Mobile') ]
df_DATA_Global_4 = df_DATA_Global_4[['Date_Format_1', 'Courts']]

df_DATA_Global = pd.concat([df_DATA_Global_1, df_DATA_Global_2, df_DATA_Global_3, df_DATA_Global_4], join='outer')
t_table_DATA_Global = pd.pivot_table(df_DATA_Global , index=['Date_Format_1'],aggfunc = 'sum')
df_DATA_Global=pd.DataFrame(t_table_DATA_Global.to_records())

df_DATA_Global = pd.merge(left=df_DATA_Global, right=df_Date_Table, how='left' , on='Date_Format_1')

DN = pd.Timestamp.now().to_period('D')
DN.to_timestamp().strftime("%Y%m%d")
df_DATA_Global = df_DATA_Global[(df_DATA_Global['Date_Format_1'] < int(DN.to_timestamp().strftime("%Y%m%d")))]

df_DATA_Global.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_DATA_Global.csv' , index=False, sep = ';' , decimal=",")